In [1]:
import numpy as np,torch
from torch.utils.data import DataLoader
from pathlib import Path
from translation_machine import collate_fn_mod
from translation_machine import dataset_mod,sentence_mod

import torch,inspect
import argparse
from pathlib import Path
from functools import partial
import numpy as np
from argparse import Namespace


In [2]:
from dev import namespace_tools
import torch
path_back_up = "../models/sequence_translator_transformer_new.pth"

state_dict = torch.load(path_back_up)
notebook_run = namespace_tools.NameSpaceAggregation.load_state_dict(state_dict["notebook_runs"][-1])
notebook_run.diffuse(globals())

In [ ]:
train_state_control.load_from_backup = True


In [3]:
from ploomber_engine.ipython import PloomberClient
from ploomber import DAG
from pathlib import Path
from ploomber.products import File

# initialize client
client = PloomberClient.from_path(Path("./trainings/training_setup.ipynb"),cwd=Path("../../"))
from argparse import Namespace

from translation_machine.models import transformer_mod
from translation_machine import sentence_mod

initial_namespace_as_dict = notebook_run.diffuse()
train_setup = client.get_namespace(initial_namespace_as_dict)
for key,val in train_setup.items():
        globals()[key] = val

Executing cell: 13: 100%|███████████████████████| 21/21 [00:00<00:00, 27.18it/s]


In [4]:
from translation_machine.models import transformer_mod

model_inputs = back_up_train_state["model_inputs"]

model = transformer_mod.TransformerForSeq2Seq(**model_inputs)

model.load_state_dict(back_up_train_state["model_params"])
model = model.eval()
model = model.to("cuda")
model.training

False

In [5]:
from translation_machine import translator_mod

translator = translator_mod.Translator(model)

In [6]:
translator.model = translator.model.eval()

In [ ]:

for english_sentence,french_sentence in train_dataset:
    translation = translator(english_sentence,limit_sentence=10)
    print(f"input sentence :  {english_sentence}")
    print(f"target sentence  : {french_sentence}")
    print(f"predicted sentence {translation}")
    print("\n")

/root/miniconda/lib/python3.9/site-packages/torch/nn/modules/activation.py:1144: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at ../aten/src/ATen/native/transformers/attention.cpp:150.)
  return torch._native_multi_head_attention(


input sentence :  Go .
target sentence  : <sos> Va ! <eos>
predicted sentence <sos> ! ! ! ! ! ! ! ! ! <eos>


input sentence :  Hi .
target sentence  : <sos> Salut ! <eos>
predicted sentence <sos> ! ! ! ! ! ! ! ! ! <eos>


input sentence :  Hi .
target sentence  : <sos> Salut . <eos>
predicted sentence <sos> ! ! ! ! ! ! ! ! ! <eos>


input sentence :  Run !
target sentence  : <sos> Cours ! <eos>
predicted sentence <sos> ! ! ! ! ! ! ! ! ! <eos>


input sentence :  Run !
target sentence  : <sos> Courez ! <eos>
predicted sentence <sos> ! ! ! ! ! ! ! ! ! <eos>


input sentence :  Who ?
target sentence  : <sos> Qui ? <eos>
predicted sentence <sos> ! ! ! ! ! ! ! ! ! <eos>


input sentence :  Wow !
target sentence  : <sos> Ça alors ! <eos>
predicted sentence <sos> ! ! ! ! ! ! ! ! ! <eos>


input sentence :  Fire !
target sentence  : <sos> Au feu ! <eos>
predicted sentence <sos> ! ! ! ! ! ! ! ! ! <eos>


input sentence :  Help !
target sentence  : <sos> À l' aide ! <eos>
predicted sentence <so